In [1]:
#!chmod +x ../setup.sh
#!cd .. && bash setup.sh VertexAI

In [2]:
from langchain_google_vertexai import ChatVertexAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import chain

In [3]:
from langchain_core.pydantic_v1 import BaseModel

class CompanyRevenue (BaseModel):
    name: str
    revenue: float
    net_margin: float
    gross_margin: float


/Users/rgutha/venvs/jupyter_env/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3667: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
template = ChatPromptTemplate.from_messages([
    ('system', '''
        You are an investor trying to fetch the financial details of the publicly traded company.
    '''),
    ('human', 'Company Ticker: {ticker}'),
    ('human', 'Quarter: {quarter}'),
    ('human', 'Question: {question}')
])

In [5]:
# Load the Gemini model from Vertex AI
llm = ChatVertexAI(
    model_name="gemini-2.0-flash-001",
    temperature=1
)

In [6]:
@chain
def chatbot (values):
    prompt = template.invoke(values)
    structured_llm = llm.with_structured_output(CompanyRevenue)
    return structured_llm.invoke (prompt)

In [7]:
response = chatbot.invoke ({
    "ticker": "MANH",
    "quarter": "2024 Q3",
    "question": "Need these details metrics - name, revenue, net margin, gross margin and margins"
})
response

CompanyRevenue(name='MANH', revenue=1000000.0, net_margin=0.2, gross_margin=0.1)